# Hyperparameter Optimization

The aim of this notebook is to tune the hyperparameters for the 3 models that were chosen to be included in the ensembling: 
- [Inception](https://arxiv.org/abs/1409.4842) & [Unet++](https://arxiv.org/pdf/1807.10165.pdf)
- [SegFormer](https://arxiv.org/pdf/2105.15203.pdf) & [Unet](https://arxiv.org/abs/1505.04597)
- [EfficientNet](https://arxiv.org/abs/1905.11946) & [Unet++](https://arxiv.org/pdf/1807.10165.pdf)

[Flaml](https://arxiv.org/pdf/1911.04706.pdf) is used for that purpose as it offers a convenient API and [efficient optimization algorithm](https://arxiv.org/pdf/2005.01571.pdf).

## Google Colab

The following two cells will only be necessary in Google Colab. To avoid problems with imports, they are included in the notebook.

In [1]:
import json
import sys

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    # noinspection PyUnresolvedReferences
    from google.colab import drive
    drive.mount('/content/drive')

In [2]:
import os
import glob

# let's keep this cell at the beginning for every notebook
# for more convenient training in Google Colab
def get_root_path(filename: str) -> str: 
    """Get root path based on notebook's name."""
    filepath = glob.glob(os.getcwd() + '/**/' + filename, recursive = True)[0]
    return os.path.dirname(os.path.dirname(filepath))

ROOT_PATH = get_root_path('hyperopt.ipynb')
sys.path.append(ROOT_PATH)

# go to the drive directory
os.chdir(ROOT_PATH) if IN_COLAB else None

## Imports

In [3]:
import os
import cv2

import albumentations as A
import flaml

from ray import tune
from scripts.preprocessing import RoadDataset, split_data
from scripts.training import setup_seed, tune_hyperparams

In [4]:
# hacky way for avoid problems with SSL when downloading some of the models
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [5]:
# setup seed for every possible way (numpy, Python, random, torch)
SEED = 16
setup_seed(SEED)

## Data

Specify the data directory and transformations. Notice that the data must first be downloaded using bash script (see README).

In [6]:
# specify train directory
train_directory = os.path.join(ROOT_PATH, 'data', 'raw', 'train')

In [7]:
# define transformations
train_tf = A.Compose([
    A.Resize(height=608, width=608, always_apply=True),
    A.Rotate(p=0.5, limit=180, border_mode=cv2.BORDER_CONSTANT, rotate_method="ellipse"),
    A.RandomBrightnessContrast(p=0.5)
])

valid_tf = A.Compose([A.Resize(height=608, width=608, always_apply=True)])

In [8]:
# split the image paths into train and validation
image_path_train, image_path_val, mask_path_train, mask_path_val = split_data(train_directory, 0.2)

# get train and val dataset instances
train_dataset = RoadDataset(image_path_train, mask_path_train, train_tf)
val_dataset = RoadDataset(image_path_val, mask_path_val, valid_tf)

# put them into a tuple for the following optimization function
ds = (train_dataset, val_dataset)

## Hyperparameters

First, specify the encoder-decoder pair that you want to optimize, and then you can specify optimization parameters. For the parameters, we optimize:

- learning rate ("lr")
- epochs ("num_epochs")
- batch_size ("batch_size")
- loss function: either DiceLoss or FocalLoss ("criterion")

In [14]:
# specify the encoder-decoder pair
ENCODER = 'efficientnet-b4'
DECODER = 'UnetPlusPlus'

In [15]:
# optimization parameters
max_num_epoch = 150
time_budget_s = 9000  # 2.5 hours 
num_samples = -1  # as long as the time budget allows

config = {
    "lr": tune.loguniform(1e-4, 1e-1),
    "num_epochs": tune.loguniform(1, max_num_epoch),
    "batch_size": tune.randint(1, 9),
    "criterion": tune.choice(["dice_loss", "focal_loss"])
}

## Tuning

Now we can use Flaml with Ray and Optuna to optimize the hyperparameters.

In [ ]:
result = flaml.tune.run(
    tune.with_parameters(tune_hyperparams, encoder=ENCODER, decoder=DECODER, datasets=ds), 
    config=config,
    metric="f1",
    mode="max",
    low_cost_partial_config={"num_epochs": 30},
    resources_per_trial={'gpu': 1},
    max_resource=max_num_epoch,
    local_dir='logs/',
    time_budget_s=time_budget_s,
    num_samples=num_samples,
    use_ray=True
)

## Save Best Config

Best config will be saved to JSON.

In [25]:
best_config = result.get_best_trial("f1", "max", "all").config.copy()
best_config

In [27]:
# specify config path and read it in
config_path = os.path.join(ROOT_PATH, 'data', 'results', 'hyperopt', 'configs.json')
with open(config_path, 'r') as file:
  data = json.load(file)

# as JSON allows only strings as keys -> generate encoder-decoder string
model = "+".join([ENCODER, DECODER])
data[model] = best_config

In [20]:
data

{'resnet18+unet': {'num_epochs': 1.0,
  'lr': 0.00019601811957324142,
  'batch_size': 1}}

In [29]:
# save the updated JSON
with open(config_path, 'w') as file:
  json.dump(data, file)